# Load the model

In [1]:
from azureml.core import Workspace
ws = Workspace.get(name='demo-aml',subscription_id='YOUR-SUSCRIPTION-ID',resource_group='demo-aml')

In [2]:
#List last model registered on ws in order to create the image
from azureml.core import Model
model_name = 'BABY-WEIGHT'
list_models = Model.list(workspace=ws, name=model_name)
model = list_models[0]
model

Model(workspace=Workspace.create(name='demo-aml', subscription_id='92ba1152-3fff-41db-864f-d94ce03eb9e1', resource_group='demo-aml'), name=BABY-WEIGHT, id=BABY-WEIGHT:8, version=8, tags={'Model': 'BABY-WEIGHT', 'Type': 'Neural Network', 'User': 'alejandra.taborda-londono@capgemini.com'}, properties={})

# Create inference configuration

In [3]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags=model.tags, 
                                               description=model.description)

In [5]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

myenv = Environment.get(workspace=ws, name="keras-env")
inference_config = InferenceConfig(entry_script="./score.py", environment=myenv)


# Deploy web service

In [6]:
from azureml.core.webservice import Webservice
service_name = 'baby-weights-webservice-aci' 

try:
    Webservice.check_for_existing_webservice(name=service_name, workspace=ws)
    service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)
    print(service.state)
    print(service.wait_for_deployment(show_output=True))

except Exception as e: 
    #print(e)
    service = Webservice(name=service_name, workspace=ws)
    # Update to new model(s).
    service.update(models=[model], inference_config=inference_config)
    service.wait_for_deployment(show_output=True)
    
    print(service.state)
    print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-21 21:25:18+00:00 Creating Container Registry if not exists.
2021-07-21 21:25:19+00:00 Registering the environment.
2021-07-21 21:25:20+00:00 Use the existing image.
2021-07-21 21:25:20+00:00 Generating deployment configuration.
2021-07-21 21:25:21+00:00 Submitting deployment to compute.
2021-07-21 21:25:24+00:00 Checking the status of deployment baby-weights-webservice-aci..
2021-07-21 21:25:35+00:00 Checking the status of inference endpoint baby-weights-webservice-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
2021-07-21T21:25:27,214783493+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-07-21T21:25:27,215676198+00:00 - iot-server/run 
2021-07-21T21:25:27,218476212+00:00 - rsyslog/run 
2021-07-21T21:25:27,219450017+00:0

In [ ]:
%%time

service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)
print(service.state)
print(service.wait_for_deployment(show_output=True))

## Test the endpoint

In [7]:
import pandas as pd
X_val=pd.read_json('inputs/X_validation_data.json', orient="split")
X_val.head()

X_val = X_val.values

In [9]:
import numpy as np
import requests

# Online prediction: send a random row from the validatition set to score
random_index = np.random.randint(0, len(X_val)-1)
input_data = "{\"data\": [" + str(list(X_val[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("input data:", input_data)
print("prediction:", resp.text)

POST to url http://4d3373d3-979c-473a-b9b1-22d9458183d2.westeurope.azurecontainer.io/score
input data: {"data": [[1.0279457598, 0.41701790990000004, 0.34644487620000003, 0.1509989142, -0.1235973895, -0.9249775915, -0.10685439490000001, -0.4628319709, 1.3865730038000001, 0.9039395828000001, -0.33874908130000003, -0.3531535229, -0.2438582988, -0.026721312400000002, -0.1921349527, -0.0653348063, -0.10667954090000001, 5.4075479387, -0.1140049685, -0.1225357703, -0.0414599321, -0.11815008090000001, -0.22951098050000002, -0.061816280200000004, -0.0612510832, -0.0755838971, -0.0945258989, -0.046841785600000005, -0.0615746786, -0.0587682023]]}
prediction: [[3.5170540809631348]]
